In [3]:
cd ..

SyntaxError: invalid syntax (2078655017.py, line 1)

In [4]:
pwd

'/Users/wliao0504/code/clif/CLIF-MIMIC'

In [38]:
import pandas as pd
import duckdb

In [12]:
from src.utils import clif_table_pathfinder, mimic_table_pathfinder

In [ ]:
pos = pd.read_parquet(clif_table_pathfinder("position"))
pos.value_counts(["position_name", "position_category"], dropna=False)

In [29]:
pos.recorded_dttm.min(), pos.recorded_dttm.max()

(Timestamp('2109-07-25 05:00:00+0000', tz='UTC'),
 Timestamp('2214-07-26 19:00:00+0000', tz='UTC'))

In [31]:
pos.sort_values("recorded_dttm", ascending=True)

,hospitalization_id,recorded_dttm,position_name,position_category
3033625,20880438,2109-07-25 05:00:00+00:00,Left Side,not_prone
95783,21838901,2109-08-22 05:00:00+00:00,Rotating,not_prone
2535900,24305596,2110-01-11 17:40:00+00:00,Supine,not_prone
2535901,24305596,2110-01-11 19:30:00+00:00,Left Side,not_prone
2535902,24305596,2110-01-11 21:00:00+00:00,Supine,not_prone
...,...,...,...,...
1622870,28006722,2214-07-26 11:00:00+00:00,Rotating,not_prone
1622876,28006722,2214-07-26 13:00:00+00:00,Rotating,not_prone
1622877,28006722,2214-07-26 15:00:00+00:00,Rotating,not_prone
1622878,28006722,2214-07-26 17:00:00+00:00,Sitting,not_prone


In [13]:
hosp = pd.read_parquet(clif_table_pathfinder("hospitalization"))

In [14]:
hosp.columns

Index(['patient_id', 'hospitalization_id', 'hospitalization_joined_id',
       'admission_dttm', 'discharge_dttm', 'age_at_admission',
       'admission_type_name', 'admission_type_category', 'discharge_name',
       'discharge_category', 'zipcode_nine_digit', 'zipcode_five_digit',
       'census_block_code', 'census_block_group_code', 'census_tract',
       'state_code', 'county_code'],
      dtype='object')

In [34]:
mimic_pat = pd.read_parquet(mimic_table_pathfinder("patients"))

In [20]:
mimic_pat

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,None
2,10000058,F,33,2168,2020 - 2022,None
3,10000068,F,19,2160,2008 - 2010,None
4,10000084,M,72,2160,2017 - 2019,2161-02-13
...,...,...,...,...,...,...
364622,19999828,F,46,2147,2017 - 2019,None
364623,19999829,F,28,2186,2008 - 2010,None
364624,19999840,M,58,2164,2008 - 2010,2164-09-17
364625,19999914,F,49,2158,2017 - 2019,None


In [27]:
mimic_pat.anchor_year.min(), mimic_pat.anchor_year.max()

(np.int64(2110), np.int64(2208))

In [32]:
mimic_pat.query("anchor_year == 2180")

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
32,10000904,F,22,2180,2011 - 2013,None
114,10002859,F,19,2180,2008 - 2010,None
208,10005475,M,20,2180,2014 - 2016,None
300,10008064,F,44,2180,2008 - 2010,None
...,...,...,...,...,...,...
364265,19990072,F,30,2180,2014 - 2016,None
364282,19990462,F,65,2180,2014 - 2016,None
364286,19990563,M,71,2180,2014 - 2016,None
364437,19994772,F,54,2180,2011 - 2013,2181-05-17


In [66]:
query = """
SElECT
    hosp.hospitalization_id,
    recorded_dttm,
    position_name,
    position_category,
    patient_id, 
    admission_dttm,
    age_at_admission,
    subject_id,
    anchor_year,
    anchor_year_group,
    date_diff('year', make_date(anchor_year, 1, 1), recorded_dttm) as year_diff,
    CAST(anchor_year_group[:4] AS INTEGER) + year_diff as actual_start_year,
    CAST(anchor_year_group[:4] AS INTEGER) + year_diff + 2 as actual_end_year
FROM pos 
LEFT JOIN hosp USING (hospitalization_id)
LEFT JOIN mimic_pat pat ON hosp.patient_id = pat.subject_id
ORDER BY actual_start_year, hospitalization_id
"""
df = duckdb.sql(query).df()
print(len(pos), len(hosp), len(mimic_pat), len(df))
df.head()

3157996 546028 364627 3157996


,hospitalization_id,recorded_dttm,position_name,position_category,patient_id,admission_dttm,age_at_admission,subject_id,anchor_year,anchor_year_group,year_diff,actual_start_year,actual_end_year
0,20001361,2143-05-09 22:00:00-06:00,Supine,not_prone,14577567,2143-05-04 13:55:00-06:00,30,14577567,2143,2008 - 2010,1,2009,2011
1,20001361,2143-05-10 06:54:00-06:00,Supine,not_prone,14577567,2143-05-04 13:55:00-06:00,30,14577567,2143,2008 - 2010,1,2009,2011
2,20001361,2143-05-09 20:00:00-06:00,Left Side,not_prone,14577567,2143-05-04 13:55:00-06:00,30,14577567,2143,2008 - 2010,1,2009,2011
3,20001361,2143-05-04 16:23:00-06:00,Supine,not_prone,14577567,2143-05-04 13:55:00-06:00,30,14577567,2143,2008 - 2010,1,2009,2011
4,20001361,2143-05-05 03:53:00-06:00,Supine,not_prone,14577567,2143-05-04 13:55:00-06:00,30,14577567,2143,2008 - 2010,1,2009,2011


In [57]:
df.actual_start_year.min(), df.actual_start_year.max()

(np.int64(2009), np.int64(2024))

In [ ]:
# in the current CLIF-MIIMC position table
df.value_counts(["position_name", "position_category"], dropna=False)

position_name           position_category
Right Side              not_prone            870992
Supine                  not_prone            865194
Left Side               not_prone            857298
Rotating                not_prone            390553
Sitting                 not_prone            157498
Prone                   prone                  8425
Reverse T-berg          not_prone              3147
Swimmers Position - L   not_prone              2591
Swimmers Position - R   not_prone              2298
Name: count, dtype: int64

In [72]:
# actual_start_year = 2021 would mean any time in the 2021-2023 window that MIMIC has blurred 
prone = df.query("position_category == 'prone'")
prone.value_counts(["actual_start_year"], dropna=False, sort=False)

actual_start_year
2009                  470
2010                  160
2011                   96
2012                  442
2013                  117
2014                   67
2015                  376
2016                  104
2017                   61
2018                  691
2019                  385
2020                  571
2021                 4292
2022                  469
2023                  124
Name: count, dtype: int64

In [73]:
mimic_pat.query("subject_id == 14577567")

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
166128,14577567,M,30,2143,2008 - 2010,None
